In [1]:
!pip install -q transformers accelerate bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 29.2 MB/s eta 0:00:00


In [2]:
import pandas as pd
import torch
from transformers import LlavaProcessor, LlavaForConditionalGeneration, BitsAndBytesConfig
from PIL import Image
from tqdm.auto import tqdm

2025-05-16 22:12:38.159508: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747433558.600954      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747433558.727400      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
val_df = pd.read_csv("/kaggle/input/vqa-test-dataset/blip_vqa_val.csv")
test_df = pd.read_csv("/kaggle/input/vqa-test-dataset/blip_vqa_test.csv")
data_df = pd.concat([val_df, test_df], ignore_index=True)

data_df = data_df.head(100)

In [4]:
model_name = "llava-hf/llava-1.5-7b-hf"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16,
)
device = "cuda" if torch.cuda.is_available() else "cpu"

In [5]:
model = LlavaForConditionalGeneration.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    torch_dtype=torch.float16,
)
processor = LlavaProcessor.from_pretrained(model_name)

config.json:   0%|          | 0.00/950 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/70.1k [00:00<?, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/3.62M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/41.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/552 [00:00<?, ?B/s]

processor_config.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/701 [00:00<?, ?B/s]

In [6]:
def generate_one(image_path: str, question: str, max_new_tokens: int = 30) -> str:
    img = Image.open(image_path).convert("RGB")
    prompt = (
        "### Instruction:\n"
        "<image>\n"
        f"Answer the question in exactly one word: {question}\n"
        "### Response:"
    )

    # 1) Tokenize + run
    inputs = processor(images=img, text=prompt, return_tensors="pt") \
                      .to(device, non_blocking=True)
    outputs = model.generate(**inputs, max_new_tokens=max_new_tokens)

    # 2) Decode full text
    raw = processor.tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]

    # 3) Remove everything up through the response marker
    if "### Response:" in raw:
        answer_section = raw.split("### Response:")[-1].strip()
    else:
        answer_section = raw.strip()

    # 4) Finally, grab only the first word
    return answer_section.split()[0]


In [7]:
records = []
for row in tqdm(data_df.itertuples(index=False), total=len(data_df), desc="Running LLaVA"):
    img_path = f"/kaggle/input/abo-small/images/small/{row.image_path}"
    q_text   = row.question
    pred     = generate_one(img_path, q_text)
    gt       = getattr(row, "answer", None)
    records.append({
        "question":     q_text,
        "image_path":   row.image_path,
        "ground_truth": gt,
        "prediction":   pred
    })

Running LLaVA:   0%|          | 0/100 [00:00<?, ?it/s]

In [8]:
pred_df = pd.DataFrame(records)
pred_df.to_csv("llava_baseline_predictions.csv", index=False)

In [9]:
pred_df

,question,image_path,ground_truth,prediction
0,What color is the necklace?,76/76197bf4.jpg,Gold,Gold
1,What shape is the pendant?,76/76197bf4.jpg,Lotus,Round
2,What are the accents made of?,76/76197bf4.jpg,Crystals,Diamond
3,What does the pendant resemble?,76/76197bf4.jpg,Flower,Flower
4,What type of band is it?,76/76197bf4.jpg,Chain,Gold
...,...,...,...,...
95,What shape are the stones?,12/12cdcbd4.jpg,Round,Round
96,How many earrings are there?,12/12cdcbd4.jpg,Two,2
97,What is the item?,12/12cdcbd4.jpg,Earrings,Ring
98,What is the base material?,12/12cdcbd4.jpg,Silver,Gold
